In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
os.environ["MUJOCO_GL"] = "egl"

from PIL import Image
import numpy as np
import asyncio
import requests
import time

from IPython import display

%matplotlib inline
import matplotlib.pyplot as plt

import metaworld
import random
from llava.constants import DEFAULT_IMAGE_TOKEN

from llserver.utils import put_task, get_task_result

In [5]:
env_name = 'pick-place-v2' # Pick an environment name
render_mode = 'rgb_array' # set a render mode

ml1 = metaworld.ML1(env_name) # Construct the benchmark, sampling tasks
camera_name = 'corner' # one of: ['corner', 'corner2', 'corner3', 'topview', 'behindGripper', 'gripperPOV']
env = ml1.train_classes[env_name](render_mode=render_mode, camera_name=camera_name)

task = random.choice(ml1.train_tasks)
env.set_task(task)  # Set task
obs = env.reset()  # Reset environment

In [6]:
for i in range(10):
    action = np.array([1, 0, 0, 0])
    (next_obs, reward, terminated, truncated, info) = env.step(action)


In [ ]:
img1 = Image.fromarray(env.render()[::-1,::,::], 'RGB')
img1.save('metatmp1.png')
img1

In [11]:
prompt = f"""
You an action planner of a robot arm.
You are able to execute commands:
- move up
- move down
- move left
- move right
- move back
- move forward
- grasp object

You are provided with two images of a different state of execution.
Tell what commands were executed.
image 1: {DEFAULT_IMAGE_TOKEN} 
image 2: {DEFAULT_IMAGE_TOKEN}
"""
img_path = "~/work/meta_world/src/data/"
task_response = put_task([img_path+"metatmp.png", img_path+"metatmp1.png"], prompt)
task_id = task_response.get("task_id")

In [ ]:
task_id

In [ ]:
result_response = get_task_result(task_id)
result_response

In [ ]:
import os
os.environ['HF_HOME'] = "~/work/meta_world/llava_ckhp"

import asyncio
import uuid
from PIL import Image
import requests
import copy
import torch
import warnings
from llava.model.builder import load_pretrained_model
from llava.mm_utils import process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.conversation import conv_templates

conv_template = "qwen_1_5"
question = "Hiii"  # Токены изображений уже содержатся в промпте
conv = copy.deepcopy(conv_templates[conv_template])
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
prompt_question = conv.get_prompt()

In [ ]:
conv_templates["qwen_1_5"]

In [ ]:
print(prompt_question)